<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/06_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06 PyTorch入門 (3)
* IMDbデータセットの感情分析をPyTorchを使っておこなう。
 * 前にscikit-learnを使って同じ作業をおこなった。
* 参考資料
 * PyTorch公式のチュートリアル https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* データは以前作ったIMDbの文書埋め込みを使う。
* sentiment analysisのもっと高度な手法については、下記リンク先を参照。
 * https://github.com/bentrevett/pytorch-sentiment-analysis

## 06-01 fastTextによる文書埋め込みをMLPの入力として使うための準備
* MLP(多層パーセプトロン)の学習ぐらいは、空気を吸ったり吐いたりするぐらい自然にできるようにしておこう。

### データファイルが置いてあるGoogle Driveのパスを変数PATHに設定
* データファイルの扱い方
 * Blackboardで「自然言語処理特論」へ行く。
 * 「教材/課題/テスト」→「data」→「IMDb」と順にクリックする。
 * 見えている６つの「.npy」ファイルをダウンロードする。
 * ダウンロードした6つのファイルを、自分のGoogle Driveの適当な場所にアップロードする。
 * 次のセルで、その置き場所を指定する。

In [ ]:
PATH = '/content/drive/MyDrive/2020Courses/NLP/'

あらかじめランタイムをGPUに設定しておこう

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

### 単語埋め込みデータファイルの読み込み

In [ ]:
texts = dict()
labels = dict()
for tag in ['train', 'valid', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
for tag in ['train', 'valid', 'test']:
  print(texts[tag].shape)

### ndarrayをPyTorchのテンソルに変換

In [ ]:
for tag in ['train', 'valid', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

## 06-02 学習のための準備

### 定数の設定

In [ ]:
TRAIN_SIZE, EMBED_DIM = texts['train'].shape
NUM_CLASS = len(torch.unique(labels['train']))

VALID_SIZE = labels['valid'].shape[0]
TEST_SIZE = labels['test'].shape[0]

In [ ]:
print((f'埋め込みの次元 {EMBED_DIM}, '
  f'クラスの数 {NUM_CLASS}, '
  f'訓練データのサイズ {TRAIN_SIZE}, '
  f'検証データのサイズ {VALID_SIZE}, '
  f'テストデータのサイズ {TEST_SIZE}'))

In [ ]:
X_train, y_train = texts['train'], labels['train']
X_valid, y_valid = texts['valid'], labels['valid']
X_test, y_test = texts['test'], labels['test']

### データローダの作成
* shuffleをTrueにして、毎エポック異なる順で訓練データを見ていくようにする。

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)

## 06-03 モデルの定義と学習の準備

In [ ]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

### 損失関数とoptimizerとschedulerを作る
* https://pytorch.org/docs/stable/optim.html

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

### 訓練用の関数
* 前回とほぼ同じ。
* データのフォーマットが変わっただけ。

In [ ]:
def train_func():

  # Train the model
  train_loss = 0.0
  train_acc = 0.0
  cnt = 0
  for i, (text, cls) in enumerate(train_loader):
    optimizer.zero_grad()
    text, cls = text.to(device), cls.to(device)
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()
    cnt += 1

  # Adjust the learning rate
  scheduler.step()

  return train_loss / cnt, train_acc / cnt

### 評価用の関数

In [ ]:
def test(X, y):
  test_loss = 0.0
  acc = 0.0
  cnt = 0
  data = DataLoader(TensorDataset(X, y), batch_size=BATCH_SIZE)
  for text, cls in data:
    text, cls = text.to(device), cls.to(device)
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      test_loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()
      cnt += 1

  return test_loss / cnt, acc / cnt

## 06-04 分類器の訓練と評価

In [ ]:
import time

N_EPOCHS = 50
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train_func()
  valid_loss, valid_acc = test(X_valid, y_valid)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print(f'Epoch: {epoch+1:3d}\t|\ttime in {mins} minutes {secs} seconds')
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(X_test, y_test)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

# 課題6
* モデルやoptimizerやschedulerを変更して、validation setの分類性能をできるだけ向上させてみよう。
* その後、自分で選択した設定を使って、最終的にtest setで評価しよう。